# Compare NorESM2-LM data with profiles from GLODAPv2.2020

1. Load GLODAPv2.2020 data
2. Import talk dataset as a xarray.Dataset object
3. Find point coordinates corresponding to GLODAP casts, and nearest corresponding grid point
3. Extract subset of talk dataset for grid point
4. Plot talk values: time vs. depth with talk color scale

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import xarray as xr
import pandas as pd
import numpy as np

In [3]:
#from geopy import distance

In [4]:
from os.path import expanduser
home_dir = expanduser("~")

In [5]:
from math import pi
from scipy.spatial import cKDTree

class KDtree_fast(object):
    # def __init__(self, ncfile, latvarname, lonvarname):
    def __init__(self, latvar, lonvar):
        self.lats = latvar
        self.lons = lonvar
        # Read latitude and longitude from file into numpy arrays
        rad_factor = pi/180.0   # for trignometry, need angles in radians
        self.latrad = self.lats[:] * rad_factor
        self.lonrad = self.lons[:] * rad_factor
        self.shape = self.latrad.shape
        # Convert lat/lon to cartesian coordinate system
        # x = cos(lat) * cos(lon) * radius
        # y = cos(lat) * sin(lon) * radius
        # z = sin(lon) * radius
        clat, clon = np.cos(self.latrad), np.cos(self.lonrad)
        slat, slon = np.sin(self.latrad), np.sin(self.lonrad)
        triples = list(
            zip(np.ravel(clat*clon), np.ravel(clat*slon), np.ravel(slat)))
        self.kdt = cKDTree(triples)

    def query(self, lat0, lon0):
        rad_factor = pi/180.0
        lat0_rad = lat0 * rad_factor
        lon0_rad = lon0 * rad_factor
        clat0, clon0 = np.cos(lat0_rad), np.cos(lon0_rad)
        slat0, slon0 = np.sin(lat0_rad), np.sin(lon0_rad)
        dist_sq_min, minindex_1d = self.kdt.query(
            [clat0*clon0, clat0*slon0, slat0])
        iy_min, ix_min = np.unravel_index(minindex_1d, self.shape)
        return iy_min, ix_min

In [6]:
# Convert mol/m3 to umol/kg
talkconv = 1. / 1.028e-3

# 1. Import GLODAP data

In [7]:
glodap_file = home_dir + '/data/data/obsdata/GLODAPv2.2020/GLODAPv2.2020_talk.csv'
df = pd.read_csv(glodap_file, na_values=-9999)
df

,year,month,day,hour,minute,latitude,longitude,depth,talk
0,1992,10,7,0,0,-56.994,-23.3090,20.0,2295.0
1,1992,10,7,0,0,-56.994,-23.3090,39.0,2290.0
2,1992,10,7,0,0,-56.994,-23.3090,39.0,2290.0
3,1992,10,7,0,0,-56.994,-23.3090,58.0,2288.0
4,1992,10,7,0,0,-56.994,-23.3090,58.0,2288.0
...,...,...,...,...,...,...,...,...,...
358725,2018,4,5,12,18,-24.000,10.7667,2007.0,2322.4
358726,2018,4,5,12,18,-24.000,10.7667,2506.0,2327.9
358727,2018,4,5,12,18,-24.000,10.7667,3007.0,2337.5
358728,2018,4,5,12,18,-24.000,10.7667,3506.0,2344.7


In [8]:
## Drop data after 2014, for comparison with NorESM2-LM historical data
df = df.drop( df[df['year'] > 2014].index)

In [9]:
## Replace columns "year", "month", "day", "hour" and "minute" with a single "datetime" column
df.insert(loc=0, column='datetime', value=pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']]))
df.drop(['day', 'hour', 'minute'], axis=1, inplace=True)

In [10]:
## re-order columns, set 'depth' as column 1
## get a list of columns
cols = list(df)
cols.insert(1, cols.pop(cols.index('depth')))
## use cols order
df = df.loc[:, cols]

In [11]:
## Sort by date and depth
df.sort_values(['datetime', 'depth'], ascending=[True, True], inplace=True)
df = df.reset_index(drop=True)
df

,datetime,depth,year,month,latitude,longitude,talk
0,1972-09-07 00:00:00,7.0,1972,9,53.758,-33.6250,2283.0
1,1972-09-07 00:00:00,16.0,1972,9,53.758,-33.6250,2282.0
2,1972-09-07 00:00:00,24.0,1972,9,53.758,-33.6250,2285.0
3,1972-09-07 00:00:00,89.0,1972,9,53.758,-33.6250,2283.0
4,1972-09-07 00:00:00,113.0,1972,9,53.758,-33.6250,2292.0
...,...,...,...,...,...,...,...
310677,2014-12-22 23:34:00,2800.0,2014,12,-69.011,-0.0183,2354.1
310678,2014-12-22 23:40:00,2500.0,2014,12,-69.011,-0.0183,2354.9
310679,2014-12-22 23:47:00,2200.0,2014,12,-69.011,-0.0183,2354.7
310680,2014-12-22 23:51:00,2000.0,2014,12,-69.011,-0.0183,2355.2


In [12]:
## Convert umol/kg to mol/m3
#df['talk'] = df['talk']*1.028e-3

In [13]:
df_cast = df.drop_duplicates(subset=['datetime', 'latitude', 'longitude'], keep='first')
df_cast

,datetime,depth,year,month,latitude,longitude,talk
0,1972-09-07 00:00:00,7.0,1972,9,53.758,-33.6250,2283.0
29,1972-09-10 00:00:00,5.0,1972,9,47.608,-39.8920,2317.0
33,1972-09-11 00:00:00,4.0,1972,9,44.958,-42.0750,2319.0
60,1972-09-12 00:00:00,7.0,1972,9,42.000,-42.0330,2358.0
101,1972-09-15 00:00:00,9.0,1972,9,39.000,-43.9830,2382.0
...,...,...,...,...,...,...,...
310677,2014-12-22 23:34:00,2800.0,2014,12,-69.011,-0.0183,2354.1
310678,2014-12-22 23:40:00,2500.0,2014,12,-69.011,-0.0183,2354.9
310679,2014-12-22 23:47:00,2200.0,2014,12,-69.011,-0.0183,2354.7
310680,2014-12-22 23:51:00,2000.0,2014,12,-69.011,-0.0183,2355.2


In [14]:
## Round latitude and longitude to nearest whole number value
#df = df.round({'latitude': 0, 'longitude': 0})
#df_cast = df_cast.round({'latitude': 0, 'longitude': 0})

In [15]:
#df_all_points = df_cast.drop_duplicates(subset=['latitude', 'longitude'], keep='first')
#df_all_points

In [16]:
#df_cast = df.astype({"latitude": np.int32, "longitude": np.int32})
#lat_list = list(range(-90, 91))
#lon_list = list(range(-180, 181))
#cast_count = np.zeros((len(lat_list), len(lon_list)))
#for index, row in df_cast.iterrows():
#    cast_count[row['latitude'], row['longitude']] = cast_count[row['latitude'], row['longitude']] + 1

# 2. Import data as a xarray.Dataset object

In [17]:
## Monthly historical NorESM2-LM 'o2' or 'co3' files
#input_files = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/historical/Omon/o2/NorESM2-LM/r1i1p1f1/o2_Omon_NorESM2-LM_historical_r1i1p1f1_gr_*.nc'
#input_files = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/historical/Omon/co3/NorESM2-LM/r1i1p1f1/co3_Omon_NorESM2-LM_historical_r1i1p1f1_gr_*.nc'
input_files = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/historical/Omon/talk/NorESM2-LM/r1i1p1f1/talk_Omon_NorESM2-LM_historical_r1i1p1f1_gr_*.nc'
dset = xr.open_mfdataset(input_files, combine='by_coords')
title_label = 'hist'

## Yearly "historical + SSP###" NorESM-LM files 
#input_files_hist = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/historical/Oyr/o2/NorESM2-LM/r1i1p1f1/o2_Oyr_NorESM2-LM_historical_r1i1p1f1_gr_*.nc'
#input_files_hist = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/historical/Oyr/ph/NorESM2-LM/r1i1p1f1/ph_Oyr_NorESM2-LM_historical_r1i1p1f1_gr_*.nc'
#dset_hist = xr.open_mfdataset(input_files_hist, combine='by_coords')

#input_files_ssp126 = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/ssp126/Oyr/o2/NorESM2-LM/r1i1p1f1/o2_Oyr_NorESM2-LM_ssp126_r1i1p1f1_gr_*.nc'
#input_files_ssp126 = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/ssp126/Oyr/ph/NorESM2-LM/r1i1p1f1/ph_Oyr_NorESM2-LM_ssp126_r1i1p1f1_gr_*.nc'
#dset_ssp126 = xr.open_mfdataset(input_files_ssp126, combine='by_coords')
#dset = xr.combine_by_coords([ dset_hist , dset_ssp126])
#title_label = 'hist+ssp126'

#input_files_ssp585 = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/ssp585/Oyr/o2/NorESM2-LM/r1i1p1f1/o2_Oyr_NorESM2-LM_ssp585_r1i1p1f1_gr_*.nc'
#input_files_ssp585 = home_dir + '/data/NTK-data/modeldata/ETHZ_CMIP6/ssp585/Oyr/ph/NorESM2-LM/r1i1p1f1/ph_Oyr_NorESM2-LM_ssp585_r1i1p1f1_gr_*.nc'
#dset_ssp585 = xr.open_mfdataset(input_files_ssp585, combine='by_coords')
#dset = xr.combine_by_coords([ dset_hist , dset_ssp585])
#title_label = 'hist+ssp585'

#dset.attrs = dset_hist.attrs

### Print out info about the dataset

In [18]:
dset.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:             (bnds: 2, i: 360, j: 385, lev: 70, time: 1980, vertices: 4)
Coordinates:
    longitude           (j, i) float64 dask.array<chunksize=(385, 360), meta=np.ndarray>
    latitude            (j, i) float64 dask.array<chunksize=(385, 360), meta=np.ndarray>
  * lev                 (lev) float64 0.0 5.0 10.0 ... 6.25e+03 6.5e+03 6.75e+03
  * j                   (j) int32 1 2 3 4 5 6 7 ... 379 380 381 382 383 384 385
  * i                   (i) int32 1 2 3 4 5 6 7 ... 354 355 356 357 358 359 360
  * time                (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) object dask.array<chunksize=(120, 2), meta=np.ndarray>
    lev_bnds            (time, lev, bnds) float64 dask.array<chunksize=(120, 70, 2), meta=np.ndarray>
    vertices_latitude   (time, j, i, vertices) float64 dask.array<chunksize=(120, 385, 360, 4), me

### Attributes of main variable

In [19]:
eval('dset.' + dset.attrs['variable_id'] + '.attrs')

{'standard_name': 'sea_water_alkalinity_expressed_as_mole_equivalent',
 'long_name': 'Total Alkalinity',
 'comment': 'total alkalinity equivalent concentration (including carbonate, nitrogen, silicate, and borate components)',
 'units': 'mol m-3',
 'original_name': 'talklvl',
 'cell_methods': 'area: mean where sea time: mean',
 'cell_measures': 'area: areacello volume: volcello',
 'history': "2019-08-15T22:07:00Z altered by CMOR: Converted type from 'd' to 'f'."}

# 3. Define point coordinate and find nearest grid point

The test point at 23.0 degrees South and 12 degrees East is in the Benguela current, close to strong upwelling cells of the Benguela upwelling system. Calculate distances only for grid coordinates inside [lat_bbox,lon_bbox] region around ref_point.

The reference point is compared with (lat,lon) grid coordinates within the bounding box, using the distance function from geopy to identify the nearest one. The distance matrix is initialized with the same size as the (lat,lon) coordinate matrices, and a value expected to be larger than the minimum distance we want to find.

In [20]:
latvals = dset.latitude.values
lonvals = dset.longitude.values
ns = KDtree_fast(latvals, lonvals)

In [28]:
[ref_year, ref_month, ref_lat, ref_lon] = df.loc[[0], ['year', 'month', 'latitude', 'longitude']].values[0].tolist()
ref_date = "{0:04d}-{1:02d}".format(int(ref_year), int(ref_month))
[ref_date, ref_lat, ref_lon]

['1972-09', 53.758, -33.625]

In [29]:
iy, ix = ns.query(ref_lat, ref_lon)
print('Closest lat lon:', ns.lats[iy, ix], ns.lons[iy, ix])

Closest lat lon: 53.90482170259432 326.32635843470234


In [30]:
print('Coords: ', dset.latitude.values[iy, ix], dset.longitude.values[iy, ix])

Coords:  53.90482170259432 326.32635843470234


In [31]:
j_value = iy + 1
i_value = ix + 1
subset = dset.sel(j=j_value, i=i_value, time=ref_date)

In [32]:
subset

<xarray.Dataset>
Dimensions:             (bnds: 2, lev: 70, time: 1, vertices: 4)
Coordinates:
    longitude           float64 dask.array<chunksize=(), meta=np.ndarray>
    latitude            float64 dask.array<chunksize=(), meta=np.ndarray>
  * lev                 (lev) float64 0.0 5.0 10.0 ... 6.25e+03 6.5e+03 6.75e+03
    j                   int32 299
    i                   int32 77
  * time                (time) object 1972-09-16 00:00:00
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    lev_bnds            (time, lev, bnds) float64 dask.array<chunksize=(1, 70, 2), meta=np.ndarray>
    vertices_latitude   (time, vertices) float64 dask.array<chunksize=(1, 4), meta=np.ndarray>
    vertices_longitude  (time, vertices) float64 dask.array<chunksize=(1, 4), meta=np.ndarray>
    talk                (time, lev) float32 dask.array<chunksize=(1, 70), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               CMIP
    branch_method:             Hybrid-restart from year 1600-01-01 of piControl
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     430335.0
    contact:                   Please send any requests or bug reports to nor...
    creation_date:             2019-08-15T22:07:00Z
    data_specs_version:        01.00.31
    experiment:                all-forcing simulation of the recent past
    experiment_id:             historical
    external_variables:        areacello volcello
    forcing_index:             1
    frequency:                 mon
    further_info_url:          https://furtherinfo.es-doc.org/CMIP6.NCC.NorES...
    grid:                      tripolar grid with 1deg nominal resolution, in...
    grid_label:                gr
    history:                   2019-08-15T22:07:00Z ; CMOR rewrote data to be...
    initialization_index:      1
    institution:               NorESM Climate modeling Consortium consisting ...
    institution_id:            NCC
    mip_era:                   CMIP6
    model_id:                  NorESM2-LM
    nominal_resolution:        100 km
    parent_activity_id:        CMIP
    parent_experiment_id:      piControl
    parent_mip_era:            CMIP6
    parent_source_id:          NorESM2-LM
    parent_sub_experiment_id:  none
    parent_time_units:         days since 0421-01-01
    parent_variant_label:      r1i1p1f1
    physics_index:             1
    product:                   model-output
    realization_index:         1
    realm:                     ocnBgchem
    run_variant:               N/A
    source:                    NorESM2-LM (2017): \naerosol: OsloAero\natmos:...
    source_id:                 NorESM2-LM
    source_type:               AOGCM
    sub_experiment:            none
    sub_experiment_id:         none
    table_id:                  Omon
    table_info:                Creation Date:(24 July 2019) MD5:08e314340b9dd...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               talk
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/1cea4b29-57fe-4379-8a68-ad674d8be2a2

In [ ]:
#iy, ix = ns.query(-23.0, -12.0)
#print('Closest lat lon:', ns.lats[iy, ix], ns.lons[iy, ix])

In [ ]:
#lat = dset.latitude.to_pandas()
#lon = dset.longitude.to_pandas()
#dist = np.ones((dset.j.size, dset.i.size)) * 1.e6
#print(np.abs(lat - 23.0))

In [ ]:
for index, row in df_all_points.iterrows():
    print(index)
    ref_point = (row['latitude'], row['longitude'])
    # drop casts that do not correspond to ref_point
    df_point = df[ (df['latitude'] == ref_point[0]) & (df['longitude'] == ref_point[1]) ]
    # Find index for nearest point
    
    for j in np.arange(dset.j.size):
        for i in np.arange(dset.i.size):
            if abs(lat.iloc[j,i] - ref_point[0]) < lat_bbox and \
            abs(lon.iloc[j,i] - ref_point[1]) < lon_bbox :
                grid_point = (lat.iloc[j,i], lon.iloc[j,i])
                dist[j,i] = distance.distance(ref_point, grid_point).km
    (j_index, i_index) = np.where(dist == dist.min())
    j_value = j_index[0] + 1
    i_value = i_index[0] + 1
    [lat_grd, lon_grd] = [lat.loc[j_value,i_value], lon.loc[j_value,i_value]]
    # Slice xarray data
    subset = dset.sel(j=j_value, i=i_value, time=slice("1972-01-01", "2015-01-01"))

In [ ]:
#ref_point = (-23.0, 12.0)
ref_point = (39.0, 137.0)
#ref_point = (65.0, 0.0)
[lat_bbox, lon_bbox] = [2.0, 2.0]

Drop casts that do not correspond to ref_point.

In [ ]:
df_point = df[ (df['latitude'] == ref_point[0]) & (df['longitude'] == ref_point[1]) ]
df_point

df_cast_point = df_cast[ (df_cast['latitude'] == ref_point[0]) & (df_cast['longitude'] == ref_point[1]) ]
df_cast_point

Extract latitude and longitude grid coordinates as pandas dataframes, for comparison with reference point.

In [ ]:
for j in np.arange(dset.j.size):
    for i in np.arange(dset.i.size):
        if abs(lat.iloc[j,i] - ref_point[0]) < lat_bbox and \
        abs(lon.iloc[j,i] - ref_point[1]) < lon_bbox :
            grid_point = (lat.iloc[j,i], lon.iloc[j,i])
            dist[j,i] = distance.distance(ref_point, grid_point).km

The (j,i) indices corresponding to the minimum distance between a grid point and the reference point must be shifted by 1 to get the (j,i) table values, because python arrays start at [0,0] whereas the table values start at [1,1].

In [ ]:
(j_index, i_index) = np.where(dist == dist.min())
j_value = j_index[0] + 1
i_value = i_index[0] + 1
print('Nearest grid point is ', dist.min(), ' km from ref_point, with (j,i) indices ', j_index, ', ', i_index, ' and values ', j_value, ',', i_value )

In [ ]:
[lat_grd, lon_grd] = [lat.loc[j_value,i_value], lon.loc[j_value,i_value]]
print('Nearest grid point is located at (', lat_grd, ',', lon_grd, ')')

# 4. Extract subset of talk dataset

subset = dset.sel(j=j_value, i=i_value, time=slice("1972-01-01", "2015-01-01"))

subset.data_vars

Convert the time variable from 'cftime' format used by xarray to 'datetime' format that can be used by matplotlib.

datetimeindex = subset.indexes['time'].to_datetimeindex()
subset['time'] = datetimeindex

data = eval('subset.' + dset.attrs['variable_id'] + '.to_pandas()')

data = data.swapaxes(0,1)

data

# 5. Plot dataset

y = -data.index.values
x = data.columns.values
X,Y = np.meshgrid(x,y)

cbar_text = dset.attrs['variable_id']
cbar_units = eval('dset.' + dset.attrs['variable_id'] + '.attrs[\'units\']')
if cbar_units != '1':
    cbar_text = cbar_text + ' [' + cbar_units + ']'
title_text = eval('dset.' + dset.attrs['variable_id'] + '.attrs[\'long_name\']') + ' : {0:7.2f} N, {1:7.2f} E : {2!s}'.format(lat_grd,lon_grd,title_label)

plt.figure(figsize=(8, 6), dpi=120, facecolor='w', edgecolor='k')
plt.contourf(X, Y, data, 10, cmap='plasma')
cbar = plt.colorbar()
cbar.set_label(cbar_text)
plt.xlabel('Time')
plt.ylabel('Depth [m]')
plt.title(title_text)

plt.figure(figsize=(8, 6), dpi=120, facecolor='w', edgecolor='k')
plt.contourf(X, Y, data, 10, cmap='plasma')
plt.ylim(-1000.0 , 0.0)
cbar = plt.colorbar()
cbar.set_label(cbar_text)
plt.xlabel('Time')
plt.ylabel('Depth [m]')
plt.title(title_text)

plt.figure(figsize=(8, 6), dpi=120, facecolor='w', edgecolor='k')
plt.scatter(df_point['talk'], -df_point['depth'])
plt.plot(data['1993-11-16 00:00:00'], y)